# GuiderROIs Test

## Test Scenarios
- **DM Stack with Butler**: Full functionality with Butler repository
- **DM Stack no Butler**: Full functionality using files


## Configuration

Set your testing preferences here


In [1]:
# ============================================================================
# CONFIGURATION - Modify these settings for your test scenario
# ============================================================================

# Data source option
USE_EXISTING_DATA = False  # Set to True use existing data

# Repository paths (modify as needed)
REPO_PATH = "data/monster_guide_repo"
#REPO_PATH = "healpix_catalog_repo"
CATALOG_PATH = "data/Monster_guide"
VIGNETTING_FILE = "data/vignetting_vs_angle.npz"

# Collection and dataset names (modify as needed)
COLLECTION = "monster_guide"
#COLLECTION = "healpix_catalogs_v1"
DATASET_NAME = "monster_guide_catalog"

# HEALPix configuration
NSIDE = 32

# Target coordinates for consistent generation and testing
TARGET_RA = 120.0   # degrees - good test coordinate  
TARGET_DEC = -45.0  # degrees - accessible from Chile

print("🔧 Test Configuration:")
print(f"  Use existing data: {USE_EXISTING_DATA}")
print(f"  Butler repo: {REPO_PATH}")
print(f"  Catalog path: {CATALOG_PATH}")
print(f"  Vignetting file: {VIGNETTING_FILE}")
print(f"  Collection: {COLLECTION}")
print(f"  Dataset: {DATASET_NAME}")
print(f"  HEALPix nside: {NSIDE}")
print(f"  Target coordinates: RA={TARGET_RA}°, Dec={TARGET_DEC}°")


🔧 Test Configuration:
  Use existing data: False
  Butler repo: data/monster_guide_repo
  Catalog path: data/Monster_guide
  Vignetting file: data/vignetting_vs_angle.npz
  Collection: monster_guide
  Dataset: monster_guide_catalog
  HEALPix nside: 32
  Target coordinates: RA=120.0°, Dec=-45.0°


## Setup and Imports


In [2]:
# Patch BestEffortIsr to use the notebook-created Butler (import module by path)
from pathlib import Path
import importlib.util
from lsst.daf.butler import Butler

# Load local guider_roi module by file path to avoid package import issues
repo_root = Path.cwd().parent
print(f"repo_root {repo_root}")
module_path = repo_root / "python/lsst/ts/observatory/control/utils/extras/guider_roi.py"
print(f"module_path {module_path}")
spec = importlib.util.spec_from_file_location("gr", str(module_path))
gr = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gr)

# Ensure flags allow BestEffortIsr path
gr.DM_STACK_AVAILABLE = True
gr.BEST_EFFORT_ISR_AVAILABLE = True

# Ensure repo exists before opening it
if not Path(REPO_PATH).exists():
    Butler.makeRepo(REPO_PATH)
    
# Build Butler for the repo created/used in this notebook
nb_butler = Butler(REPO_PATH)

class _NotebookBestEffortIsr:
    def __init__(self):
        self.butler = nb_butler

# Monkeypatch the module symbol
gr.BestEffortIsr = _NotebookBestEffortIsr
print("✅ Patched BestEffortIsr to use notebook Butler:", REPO_PATH)


repo_root /home/saluser/ts_repos/ts_observatory_control
module_path /home/saluser/ts_repos/ts_observatory_control/python/lsst/ts/observatory/control/utils/extras/guider_roi.py
✅ Patched BestEffortIsr to use notebook Butler: data/monster_guide_repo


In [3]:
import os
import sys
import warnings
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import healpy as hp

print("🔍 Checking Dependencies:")
print("=" * 40)

# Basic dependencies
try:
    import healpy as hp
    from astropy.coordinates import angular_separation
    from astropy.table import Table, vstack
    from scipy.interpolate import make_interp_spline
    BASIC_DEPS_AVAILABLE = True
    print("✅ Basic deps (healpy, astropy, scipy)")
except ImportError as e:
    BASIC_DEPS_AVAILABLE = False
    print(f"❌ Basic deps not available: {e}")

try:
    from lsst.daf.butler import Butler, ButlerConfig
    from lsst.daf.butler.registry import MissingCollectionError, NoDefaultCollectionError
    
    import lsst.geom as geom
    from lsst.afw import cameraGeom
    from lsst.geom import Angle, Extent2I, SpherePoint
    from lsst.obs.base import createInitialSkyWcsFromBoresight
    from lsst.obs.lsst import LsstCam
    from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
    
    DM_STACK_AVAILABLE = True
    print("✅ DM Stack (includes Butler)")
except ImportError as e:
    DM_STACK_AVAILABLE = False
    print(f"❌ DM Stack not available: {e}")


# BestEffortIsr
try:
    from lsst.summit.utils import BestEffortIsr
    BEST_EFFORT_ISR_AVAILABLE = True
    print("✅ BestEffortIsr")
except ImportError as e:
    BEST_EFFORT_ISR_AVAILABLE = False
    print(f"❌ BestEffortIsr not available: {e}")

# Import GuiderROIs and check its dependency flags
try:
    from lsst.ts.observatory.control.utils.extras.guider_roi import (
        GuiderROIs,
        BASIC_DEPS_AVAILABLE as GROI_BASIC,
        DM_STACK_AVAILABLE as GROI_DM_STACK,
        BEST_EFFORT_ISR_AVAILABLE as GROI_BEST_EFFORT
    )
    print("✅ GuiderROIs imported successfully")
    
    # Show GuiderROIs internal dependency detection
    print(f"\n📊 GuiderROIs Internal Detection:")
    print(f"   Basic deps: {'✅' if GROI_BASIC else '❌'}")
    print(f"   DM Stack: {'✅' if GROI_DM_STACK else '❌'}")
    print(f"   BestEffortIsr: {'✅' if GROI_BEST_EFFORT else '❌'}")
    
except ImportError as e:
    print(f"❌ Cannot import GuiderROIs: {e}")
    print("Make sure you're in the correct directory")

# Determine operational mode
print(f"\n🎯 Operational Mode:")
if BASIC_DEPS_AVAILABLE and DM_STACK_AVAILABLE:
    if BEST_EFFORT_ISR_AVAILABLE:
        print("   🟢 Summit Mode: DM Stack + BestEffortIsr (complete functionality)")
        OPERATIONAL_MODE = "summit"
    else:
        print("   🟢 Full Mode: DM Stack (complete functionality)")
        OPERATIONAL_MODE = "full"
elif BASIC_DEPS_AVAILABLE:
    print("   🟡 File Mode: Basic packages only (file-based catalogs)")
    OPERATIONAL_MODE = "file"
else:
    print("   🔴 No operational mode available")
    OPERATIONAL_MODE = "none"

print(f"\n📁 Working directory: {os.getcwd()}")


🔍 Checking Dependencies:
✅ Basic deps (healpy, astropy, scipy)
✅ DM Stack (includes Butler)
❌ BestEffortIsr not available: No module named 'lsst.summit'


current_tai uses current_tai_from_utc; clock_gettime(CLOCK_TAI) is off by 37.0 seconds


✅ GuiderROIs imported successfully

📊 GuiderROIs Internal Detection:
   Basic deps: ✅
   DM Stack: ✅
   BestEffortIsr: ❌

🎯 Operational Mode:
   🟢 Full Mode: DM Stack (complete functionality)

📁 Working directory: /home/saluser/ts_repos/ts_observatory_control/guider_roi_extra_other_branch


/home/saluser/ts_repos/ts_observatory_control/python/lsst/ts/observatory/control/utils/extras/dm_target_catalog.py:44: UserWarning: Cannot import required libraries. Module will not work.
  warnings.warn("Cannot import required libraries. Module will not work.")


In [4]:

def generate_test_data():
    """Generate test data using the external script centered in RA/Dec coordinates."""
    import subprocess
    import sys
    
    print("🔧 Generating test data using external script (RA/Dec coordinates)...")
    print(f"📍 Generating data around target coordinates: RA={TARGET_RA}°, Dec={TARGET_DEC}°")
    
    # Use the generate_test_guider_data.py script for comprehensive data generation
    # Generate data around consistent coordinates that will be used for testing
    result = subprocess.run([
        sys.executable, "generate_test_guider_data.py",  # Script is one level up from guider_roi_extras
        "--target-ra", str(TARGET_RA),      # Target coordinates for test
        "--target-dec", str(TARGET_DEC), 
        "--neighbor-radius", "2",            # Include 2 rings of neighbors  
        "--num-catalogs", "50",              # Limit total catalogs
        "--stars-per-catalog", "75",         # Good number per catalog
        "--catalog-dir", CATALOG_PATH,
        "--vignetting-file", VIGNETTING_FILE
    ], capture_output=True, text=True)
    
    print("STDOUT:")
    print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)
    
    if result.returncode == 0:
        print(f"✅ Test data generation completed successfully!")
        
        # Extract test pixels from generated files
        catalog_dir = Path(CATALOG_PATH)
        if catalog_dir.exists():
            csv_files = list(catalog_dir.glob("*.csv"))
            test_pixels = []
            for csv_file in csv_files:
                try:
                    pixel_id = int(csv_file.stem)
                    test_pixels.append(pixel_id)
                except ValueError:
                    pass
            
            test_pixels = sorted(test_pixels)
            print(f"📊 Generated {len(test_pixels)} HEALPix catalogs")
            print(f"🎯 Available pixels: {test_pixels[:10]}{'...' if len(test_pixels) > 10 else ''}")
            
            # Verify target pixel is included
            target_pixel = hp.ang2pix(NSIDE, TARGET_RA, TARGET_DEC, lonlat=True)
            if target_pixel in test_pixels:
                print(f"✅ Target pixel {target_pixel} is included in generated data")
            else:
                print(f"⚠️  Target pixel {target_pixel} not found in generated data")
            
            return test_pixels
        else:
            print("❌ Catalog directory not created")
            return []
    else:
        print(f"❌ Test data generation failed with return code: {result.returncode}")
        return []


## Data Preparation

Generate test data or check existing data based on configuration


## BestEffortIsr Integration (Summit Environment)

**Note**: This section shows how to use BestEffortIsr for automatic Butler access environments where there's a butler available. BestEffortIsr provides a butler if available.

**Usage**: Uncomment the code below if you're running in an environment where BestEffortIsr is available (e.g., summit?).


In [5]:
# ============================================================================
# BestEffortIsr Integration (Summit Environment Only)
# ============================================================================
# 
# Uncomment the code below to use BestEffortIsr for automatic Butler access


# Option 1: Let GuiderROIs automatically use BestEffortIsr
# -------------------------------------------------------
# If BestEffortIsr is available, GuiderROIs will automatically use it
# when no explicit butler or butler_config is provided
# Note: for the example below to work the (test) data should have been ingested
#       to that butler
"""
print("🏔️  Testing BestEffortIsr Integration")
print("="*50)

if BEST_EFFORT_ISR_AVAILABLE:
    try:
        # Initialize GuiderROIs without explicit Butler - it will try BestEffortIsr
        print("🔄 Initializing GuiderROIs with automatic BestEffortIsr...")
        groi_summit = GuiderROIs(
            # No butler or butler_config provided - will try BestEffortIsr automatically
            catalog_name="monster_guide_catalog",  # Adjust for your data
            collection="monster_guide",             # Adjust for your collection
            nside=32
        )
        
        print("✅ GuiderROIs initialized with BestEffortIsr Butler")
        print(f"   Butler: {groi_summit.butler}")
        print(f"   BestEffortIsr: {groi_summit.best_effort_isr}")
        
        # Test basic functionality
        if groi_summit.butler is not None:
            collections = list(groi_summit.butler.registry.queryCollections())
            print(f"   Available collections: {collections[:5]}{'...' if len(collections) > 5 else ''}")
            
            # Test ROI selection with operational data
            test_roi_selection(groi_summit)
        else:
            print("⚠️  BestEffortIsr Butler not available")
            
    except Exception as e:
        print(f"❌ BestEffortIsr integration failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  BestEffortIsr not available in this environment")
    print("   This feature is only available in summit environments")
"""

# Option 2: Explicit BestEffortIsr usage
# --------------------------------------
# For more control, you can explicitly create BestEffortIsr and pass its butler
# Note: Maybe use this method to the BestEffortIsr butler and ingest the data 
#       b4 running the tests.

"""
print("\\n🏔️  Testing Explicit BestEffortIsr Usage")
print("="*50)

if BEST_EFFORT_ISR_AVAILABLE:
    try:
        from lsst.summit.utils import BestEffortIsr
        
        # Create BestEffortIsr instance
        print("🔄 Creating BestEffortIsr instance...")
        best_effort_isr = BestEffortIsr()
        
        print("✅ BestEffortIsr created successfully")
        print(f"   Butler repo: {best_effort_isr.butler.registry.getDatasetTypes()}")
        
        # Initialize GuiderROIs with BestEffortIsr's butler
        groi_explicit = GuiderROIs(
            butler=best_effort_isr.butler,
            catalog_name="monster_guide_catalog",  # Use operational catalog name
            collection="monster_guide",             # Use operational collection
            nside=32
        )
        
        print("✅ GuiderROIs initialized with explicit BestEffortIsr Butler")
        
        # Test with operational data
        test_roi_selection(groi_explicit)
        
    except Exception as e:
        print(f"❌ Explicit BestEffortIsr usage failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  BestEffortIsr not available in this environment")
"""


'\nprint("\\n🏔️  Testing Explicit BestEffortIsr Usage")\nprint("="*50)\n\nif BEST_EFFORT_ISR_AVAILABLE:\n    try:\n        from lsst.summit.utils import BestEffortIsr\n\n        # Create BestEffortIsr instance\n        print("🔄 Creating BestEffortIsr instance...")\n        best_effort_isr = BestEffortIsr()\n\n        print("✅ BestEffortIsr created successfully")\n        print(f"   Butler repo: {best_effort_isr.butler.registry.getDatasetTypes()}")\n\n        # Initialize GuiderROIs with BestEffortIsr\'s butler\n        groi_explicit = GuiderROIs(\n            butler=best_effort_isr.butler,\n            catalog_name="monster_guide_catalog",  # Use operational catalog name\n            collection="monster_guide",             # Use operational collection\n            nside=32\n        )\n\n        print("✅ GuiderROIs initialized with explicit BestEffortIsr Butler")\n\n        # Test with operational data\n        test_roi_selection(groi_explicit)\n\n    except Exception as e:\n        pri

In [6]:

def check_existing_data():
    """Check what existing data is available."""
    print("🔍 Checking for existing data...")
    
    available_data = {
        'files': {},
        'butler_repo': {},
        'test_pixels': []
    }
    
    # Check file-based data
    print("\n📁 File-based data:")
    catalog_dir = Path(CATALOG_PATH)
    vignetting_file = Path(VIGNETTING_FILE)
    
    if catalog_dir.exists():
        csv_files = list(catalog_dir.glob("*.csv"))
        available_data['files']['catalog_files'] = len(csv_files)
        print(f"  ✅ Catalog directory: {catalog_dir} ({len(csv_files)} CSV files)")
        
        # Extract HEALPix pixel IDs from filenames
        test_pixels = []
        for csv_file in csv_files[:10]:  # Check first 10 files
            try:
                pixel_id = int(csv_file.stem)
                test_pixels.append(pixel_id)
            except ValueError:
                pass
        available_data['test_pixels'] = sorted(test_pixels)
        print(f"  📊 Available HEALPix pixels: {available_data['test_pixels'][:5]}{'...' if len(available_data['test_pixels']) > 5 else ''}")
    else:
        print(f"  ❌ Catalog directory not found: {catalog_dir}")
    
    if vignetting_file.exists():
        print(f"  ✅ Vignetting file: {vignetting_file}")
        available_data['files']['vignetting'] = True
    else:
        print(f"  ❌ Vignetting file not found: {vignetting_file}")
        available_data['files']['vignetting'] = False
    
    # Check Butler repository
    if DM_STACK_AVAILABLE:
        print("\n🗄️  Butler repository:")
        
        repo = Path(REPO_PATH)
        if repo.exists():
            try:
                butler = Butler(REPO_PATH)
                collections = list(butler.registry.queryCollections())
                datasets = list(butler.registry.queryDatasets(DATASET_NAME, collections=[COLLECTION]))
                available_data['butler_repo'] = {
                    'path': REPO_PATH,
                    'collections': collections,
                    'datasets': len(datasets)
                }
                print(f"  ✅ HEALPix repo: {repo} ({len(datasets)} datasets)")
            except Exception as e:
                print(f"  ⚠️  Repository exists but cannot access: {e}")
        else:
            print(f"  ❌ Butler repository not found: {repo}")
    else:
        print("\n🗄️  Butler repository: Skipped (DM Stack not available)")
    
    return available_data


# Execute data preparation based on configuration
if USE_EXISTING_DATA:
    print("📖 Using existing data mode")
    data_info = check_existing_data()
    
    # Use existing test pixels or fallback defaults
    if data_info['test_pixels']:
        test_pixels = data_info['test_pixels'][:5]  # Use first 5
        print(f"\n✅ Will use existing HEALPix pixels: {test_pixels}")
    else:
        test_pixels = [1000, 2000, 3000, 4000, 5000]  # Fallback defaults
        print(f"\n⚠️  No existing pixels found, using defaults: {test_pixels}")
else:
    print("🔧 Generating new data mode (using aligned coordinates)")
    test_pixels = generate_test_data()  # Use aligned version
    print("\n🎉 Test data generation complete!")

print(f"\n🎯 Test pixels to use: {test_pixels}")


🔧 Generating new data mode (using aligned coordinates)
🔧 Generating test data using external script (RA/Dec coordinates)...
📍 Generating data around target coordinates: RA=120.0°, Dec=-45.0°
STDOUT:

STDERR:
2025-08-14 22:02:16,073 - INFO - Starting test data generation...
2025-08-14 22:02:16,073 - INFO - Creating vignetting data: data/vignetting_vs_angle.npz
2025-08-14 22:02:16,076 - INFO -   - Created 51 vignetting points
2025-08-14 22:02:16,076 - INFO -   - Theta range: 0.0 to 2.0 degrees
2025-08-14 22:02:16,076 - INFO -   - Vignetting range: 0.998 to 1.000
2025-08-14 22:02:16,076 - INFO - Creating 50 monster guide catalogs in data/Monster_guide/
2025-08-14 22:02:16,077 - INFO - Targeting coordinates: RA=120.000000°, Dec=-45.000000°
2025-08-14 22:02:16,077 - INFO - Target coordinates correspond to HEALPix pixel: 10586
2025-08-14 22:02:16,077 - INFO - Found 14 pixels within radius 2 of target
2025-08-14 22:02:16,077 - INFO - Selected HEALPix pixels: [np.int64(10218), np.int64(10219),

## Testing (DM Stack) with Butler

Test GuiderROIs with HEALPix Butler and full DM Stack functionality


### Data Ingestion (Butler Repository Creation)

Create a Butler repository from the generated test data for comprehensive testing


In [7]:
def ingest_test_data():
    """Ingest the generated test data into a Butler repository."""
    import subprocess
    import sys
    
    print("🗄️  Creating Butler repository from generated test data...")
    print(f"📁 Repository path: {REPO_PATH}")
    print(f"📚 Collection: {COLLECTION}")
    print(f"📊 Dataset name: {DATASET_NAME}")
    
    # Check if we have data to ingest
    catalog_dir = Path(CATALOG_PATH)
    vignetting_file = Path(VIGNETTING_FILE)
    
    if not catalog_dir.exists():
        print(f"❌ Catalog directory not found: {catalog_dir}")
        print("   Generate test data first!")
        return False
        
    if not vignetting_file.exists():
        print(f"❌ Vignetting file not found: {vignetting_file}")
        print("   Generate test data first!")
        return False
    
    csv_files = list(catalog_dir.glob("*.csv"))
    if not csv_files:
        print(f"❌ No CSV files found in: {catalog_dir}")
        return False
        
    print(f"✅ Found {len(csv_files)} catalog files to ingest")
    print(f"✅ Found vignetting file: {vignetting_file}")
    
    # Run the ingestion script with custom dataset name to match notebook configuration
    print("\\n🔧 Running ingestion script...")
    print(f"📝 Using custom dataset name: {DATASET_NAME}")
    result = subprocess.run([
        sys.executable, "ingest_guider_data.py",
        "--repo-path", REPO_PATH,
        "--collection", COLLECTION,
        "--catalog-path", CATALOG_PATH,
        "--vignetting-file", VIGNETTING_FILE,
        "--catalog-dataset-name", DATASET_NAME,
        "--nside", str(NSIDE)
    ], capture_output=True, text=True)
    
    print("STDOUT:")
    print(result.stdout)
    if result.stderr:
        print("STDERR:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("\\n✅ Butler repository created successfully!")
        
        # Verify the repository
        try:
            if DM_STACK_AVAILABLE:
                butler = Butler(REPO_PATH)
                collections = list(butler.registry.queryCollections())
                datasets = list(butler.registry.queryDatasets(DATASET_NAME, collections=[COLLECTION]))
                
                print(f"📊 Repository verification:")
                print(f"   Collections: {collections}")
                print(f"   Datasets: {len(datasets)} catalog entries")
                print(f"   Ready for Butler-based testing!")
                return True
            else:
                print("⚠️  Cannot verify repository (DM Stack not available)")
                return True
                
        except Exception as e:
            print(f"⚠️  Repository created but verification failed: {e}")
            return True
            
    else:
        print(f"\\n❌ Ingestion failed with return code: {result.returncode}")
        return False

# Execute data ingestion if we have DM Stack and generated data
if DM_STACK_AVAILABLE and test_pixels:
    print("🗄️  Ingesting test data into Butler repository...")
    
    # Check if repository already exists
    repo = Path(REPO_PATH)
    if repo.exists():
        print(f"📁 Butler repository already exists: {repo}")
        print("   Skipping ingestion (remove directory to regenerate)")
        
        # Quick verification
        try:
            butler = Butler(REPO_PATH)
            datasets = list(butler.registry.queryDatasets(DATASET_NAME, collections=[COLLECTION]))
            print(f"✅ Found existing repository with {len(datasets)} datasets")
        except Exception as e:
            print(f"⚠️  Existing repository has issues: {e}")
            print("   Consider removing and regenerating")
    else:
        # Ingest the data
        ingestion_success = ingest_test_data()
        if not ingestion_success:
            print("❌ Failed to create Butler repository")
            print("   Butler tests will be skipped")
else:
    if not DM_STACK_AVAILABLE:
        print("⚠️  DM Stack not available - skipping Butler repository creation")
    elif not test_pixels:
        print("⚠️  No test data available - skipping Butler repository creation")
    else:
        print("⚠️  Unknown issue - skipping Butler repository creation")


🗄️  Ingesting test data into Butler repository...
📁 Butler repository already exists: data/monster_guide_repo
   Skipping ingestion (remove directory to regenerate)
✅ Found existing repository with 14 datasets


In [8]:
def test_roi_selection(groi):
    # Test parameters - choose coordinates where we have data
    # Use the target coordinates that were used for data generation
    ra_center = TARGET_RA
    dec_center = TARGET_DEC
    
    # Find the corresponding HEALPix pixel for information
    target_pixel = hp.ang2pix(NSIDE, TARGET_RA, TARGET_DEC, lonlat=True)
    
    print(f"\n📍 Test pointing:")
    print(f"   Boresight RA: {ra_center:.3f}°")
    print(f"   Boresight Dec: {dec_center:.3f}°")
    print(f"   Target HEALPix pixel: {target_pixel}")
    print(f"   Using coordinates aligned with generated data")
    
    # ROI selection parameters
    boresight_RA = ra_center
    boresight_DEC = dec_center
    boresight_RotAngle = 0.0  # degrees
    roi_size = 64  # pixels
    roi_time = 1000  # milliseconds
    band = "r"  # r-band
    
    print(f"\n🎯 Attempting ROI selection...")
    print(f"   ROI size: {roi_size}x{roi_size} pixels")
    print(f"   Integration time: {roi_time} ms")
    print(f"   Filter band: {band}")
    
    # Attempt ROI selection
    try:
        config_text, selected_stars = groi.get_guider_rois(
            ra=boresight_RA,
            dec=boresight_DEC,
            sky_angle=boresight_RotAngle,
            roi_size=roi_size,
            roi_time=roi_time,
            band=band,
            use_guider=True,
            use_science=False,
            use_wavefront=False
        )
        
        print(f"\n✅ ROI selection successful!")
        print(f"   Selected {len(selected_stars)} guide stars")
        
        # Show selected stars
        if len(selected_stars) > 0:
            print(f"\n🌟 Selected guide stars:")
            for i, star in enumerate(selected_stars):
                ra_deg = np.degrees(star['coord_ra'])
                dec_deg = np.degrees(star['coord_dec'])
                mag = star.get(f'mag_{band}', star['gaia_G'])
                print(f"   Star {i+1}: RA={ra_deg:.3f}°, Dec={dec_deg:.3f}°, mag={mag:.2f}")
                print(f"           CCD={star['ccdName']}, Amp={star['ampName']}")
            
            # Show ROI configuration (first few lines)
            print(f"\n📋 ROI Configuration (excerpt):")
            config_lines = config_text.strip().split('\n')
            for line in config_lines[:10]:  # Show first 10 lines
                print(f"   {line}")
            if len(config_lines) > 10:
                print(f"   ... ({len(config_lines)-10} more lines)")
            
            print(f"\n🎉 Full ROI selection test completed successfully!")
            
        else:
            print(f"   ⚠️  No guide stars selected (this may be normal for test data)")
            
    except RuntimeError as e:
        if "No suitable guide stars found" in str(e):
            print(f"   ⚠️  No suitable guide stars found for test pointing")
            print(f"   This is expected with limited test data coverage")
            print(f"   The algorithm is working correctly")
        else:
            print(f"   ❌ ROI selection failed: {e}")
            
    except Exception as e:
        print(f"   ❌ ROI selection failed with unexpected error: {e}")
        import traceback
        traceback.print_exc()

In [9]:
print("🧪 Testing GuiderROIs with HEALPix Butler")
print("="*55)

if not DM_STACK_AVAILABLE:
    print("❌ DM Stack not available - skipping Butler tests")
    print("   Install LSST stack to enable Butler functionality")
else:
    try:
        # Check for existing HEALPix Butler repository
        repo = Path(REPO_PATH)
        
        if repo.exists():
            print(f"✅ Found existing Butler repository: {repo}")
            
            # Connect to Butler
            butler = Butler(REPO_PATH)
            
            print(f"✅ Connected to Butler repository")
            print(f" Datase name {DATASET_NAME}")
            print(f" Collection {COLLECTION}")
            print(f"   Collections: {list(butler.registry.queryCollections())}")
            
            # Initialize GuiderROIs with HEALPix Butler
            groi_butler = gr.GuiderROIs(
                catalog_name=DATASET_NAME,
                collection=COLLECTION
            )
            print("✅ GuiderROIs initialized with HEALPix Butler")
            
            # Test vignetting correction
            print("\n📊 Testing vignetting correction:")
            test_angles = [0.0, 0.5, 1.0, 1.5, 2.0]
            for angle in test_angles:
                correction = groi_butler.vignetting_correction(angle)
                print(f"  Angle {angle:3.1f}°: Δmag = {correction:+.4f}")
            
            # Test HEALPix catalog access
            print("\n🌟 Testing HEALPix catalog access:")
            
            # Get available HEALPix pixels from the repository
            datasets = list(butler.registry.queryDatasets(
                DATASET_NAME, 
                collections=[COLLECTION]
            ))
            
            if datasets:
                print(f"  Found {len(datasets)} HEALPix catalog datasets")
                
                # Extract HEALPix pixel IDs from the datasets
                available_pixels = []
                for dataset in datasets[:10]:  # Test first 10
                    try:
                        # Get the HEALPix pixel ID from the data ID
                        healpix_dim = groi_butler.healpix_dim
                        if healpix_dim in dataset.dataId:
                            pixel_id = dataset.dataId[healpix_dim]
                            available_pixels.append(pixel_id)
                    except Exception as e:
                        print(f"    Warning: Could not extract pixel ID from {dataset}: {e}")
                
                if available_pixels:
                    test_butler_pixels = available_pixels[:3]
                    print(f"  Testing with HEALPix pixels: {test_butler_pixels}")
                    
                    # Test catalog data retrieval
                    tables = groi_butler._get_catalog_data_from_butler(test_butler_pixels)
                    print(f"  Retrieved {len(tables)} catalog tables from Butler")
                    
                    if tables:
                        total_stars = sum(len(table) for table in tables)
                        print(f"  Total stars: {total_stars}")
                        
                        # Show sample data
                        print(f"\n  Sample from first catalog:")
                        print(f"    Stars: {len(tables[0])}")
                        print(f"    Columns: {list(tables[0].colnames)}")
                        
                        # Check guide star statistics
                        isolated_counts = []
                        for table in tables:
                            isolated = table['guide_flag'] > 63
                            isolated_counts.append(isolated.sum())
                        
                        print(f"    Isolated guide stars per catalog: {isolated_counts}")
                        print(f"    Total isolated guide stars: {sum(isolated_counts)}")
                        
                        print("\n✅ HEALPix Butler test completed successfully!")

                        test_roi_selection(groi_butler)
                    else:
                        print("  ❌ No catalog data retrieved from Butler")
                else:
                    print("  ❌ Could not extract HEALPix pixel IDs from datasets")
            else:
                print("  ❌ No catalog datasets found in Butler repository")
                print(f"     Try creating data with: python ingest_guider_data.py")
                
        else:
            print(f"❌ Butler repository not found: {repo}")
            print("   Create a HEALPix Butler repository first using:")
            print(f"   python ingest_guider_data.py --repo-path {REPO_PATH} --collection {COLLECTION} --catalog-path {CATALOG_PATH}")
            
    except Exception as e:
        print(f"❌ HEALPix Butler test failed: {e}")
        import traceback
        traceback.print_exc()


🧪 Testing GuiderROIs with HEALPix Butler
✅ Found existing Butler repository: data/monster_guide_repo
✅ Connected to Butler repository
 Datase name monster_guide_catalog
 Collection monster_guide
   Collections: ['monster_guide']
✅ GuiderROIs initialized with HEALPix Butler

📊 Testing vignetting correction:
  Angle 0.0°: Δmag = -0.0000
  Angle 0.5°: Δmag = +0.0002
  Angle 1.0°: Δmag = +0.0007
  Angle 1.5°: Δmag = +0.0015
  Angle 2.0°: Δmag = +0.0026

🌟 Testing HEALPix catalog access:
  Found 14 HEALPix catalog datasets
  Testing with HEALPix pixels: [10467, 10466, 10700]
  Retrieved 3 catalog tables from Butler
  Total stars: 225

  Sample from first catalog:
    Stars: 75
    Columns: ['coord_ra', 'coord_dec', 'gaia_G', 'mag_u', 'mag_g', 'mag_r', 'mag_i', 'mag_z', 'mag_y', 'guide_flag', 'healpix_id']
    Isolated guide stars per catalog: [np.int64(55), np.int64(55), np.int64(60)]
    Total isolated guide stars: 170

✅ HEALPix Butler test completed successfully!

📍 Test pointing:
   Bor

No isolated stars found inside CCD for detector 193
No isolated stars found inside CCD for detector 194
No isolated stars found inside CCD for detector 197
No isolated stars found inside CCD for detector 198
No isolated stars found inside CCD for detector 202



✅ ROI selection successful!
   Selected 3 guide stars

🌟 Selected guide stars:
   Star 1: RA=118.353°, Dec=-43.631°, mag=13.77
           CCD=R44_SG0, Amp=C16
   Star 2: RA=122.195°, Dec=-46.099°, mag=15.53
           CCD=R00_SG1, Amp=C07
   Star 3: RA=121.716°, Dec=-46.514°, mag=14.90
           CCD=R00_SG0, Amp=C10

📋 ROI Configuration (excerpt):
   roi_spec:
    common:
     rows: 64
     cols: 64
     integration_time_millis: 1000
   roi:
    R44SG0:
     segment: 16
     start_row: 1776
     start_col: 131
   ... (10 more lines)

🎉 Full ROI selection test completed successfully!
